In [ ]:
import requests
from pyquery import PyQuery as pq
import time
import random

# 定义代理列表（可扩展）
proxies = [
    {'http': 'http://111.111.111.111:8080', 'https': 'http://111.111.111.111:8080'},
    {'http': 'http://222.222.222.222:8080', 'https': 'http://222.222.222.222:8080'},
    # 添加更多代理...
]

# 请求头设置
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'zh-CN,zh;q=0.9',
}

# 爬取电影评论的函数
def fetch_comments(movie_id, movie_name_en, movie_name_cn, max_pages=5):
    comments = []
    comment_id = 0

    for page in range(max_pages):
        start = page * 20
        url = f'https://movie.douban.com/subject/{movie_id}/comments?start={start}'
        response = requests.get(url, headers=headers, proxies=random.choice(proxies))

        if response.status_code == 200:
            doc = pq(response.text)
            comment_items = doc(".comment-item")

            for item in comment_items.items():
                comment_data = {}
                comment_data['ID'] = comment_id
                comment_data['Movie_Name_EN'] = movie_name_en
                comment_data['Movie_Name_CN'] = movie_name_cn
                comment_data['Crawl_Date'] = time.strftime("%Y-%m-%d", time.localtime())
                comment_data['Number'] = item.attr("data-id")
                comment_data['Username'] = item.find(".comment-info a").text()
                comment_data['Date'] = item.find(".comment-time").text().strip()
                stars_class = item.find(".comment-info .rating").attr("class")
                if stars_class:
                    comment_data['Star'] = int(stars_class.split("allstar")[-1][0])
                else:
                    comment_data['Star'] = None
                comment_data['Comment'] = item.find(".short").text()
                comment_data['Like'] = int(item.find(".vote-count").text() or 0)

                comments.append(comment_data)
                comment_id += 1

            # 随机延迟以防反爬虫机制
            time.sleep(random.uniform(1, 3))

        else:
            print(f"请求失败，状态码：{response.status_code}")
            break

    return comments

if __name__ == "__main__":
    movie_id = '10741834'  # 豆瓣电影ID
    movie_name_en = 'Avengers: Age of Ultron'  # 英文电影名称
    movie_name_cn = '复仇者联盟2：奥创纪元'  # 中文电影名称
    comments = fetch_comments(movie_id, movie_name_en, movie_name_cn, max_pages=5)
    for comment in comments:
        print(comment)
